In [1]:
from sklearn import preprocessing
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, LSTM, TimeDistributed, Bidirectional
from nervaluate import Evaluator
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
class txtReader:
    def __init__(self, filename):
        self.filename = filename

    def read_split(self):
        with open(self.filename, 'r') as f:
            file_read = f.read()
        lines = file_read.split('\n')
        text = []
        text_id = []
        tmp = []
        tmp_id = []
        for i in lines:
            # i = i.strip()
            if i == '':
                text.append(tmp)
                text_id.append(tmp_id)
                tmp = []
                tmp_id = []
            else:
                word = i.split('\t')
                tmp.append(word[0])
                tmp_id.append(word[1])

        return text, text_id

In [16]:
class alphabet:
    def __init__(self, train_file, dev_file, test_file):
        self.train_file = train_file
        self.dev_file = dev_file
        self.test_file = test_file
        self.data = dict()
        self.labels = dict()

    def read_split(self):
        text_files = []
        for text_file in [self.train_file, self.dev_file, self.test_file]:
            txt = txtReader(text_file)
            text, text_id = txt.read_split()
            text_files.append(text)
            text_files.append(text_id)

        return text_files[0], text_files[1], text_files[2], text_files[3], text_files[4], text_files[5]
    
    def _tagger(self, dataset, cnt, dictionary):
        for i in dataset: # i es una frase
            for j in i: # j es una palabra
                pos = i.index(j) # pos es la posicion de la palabra en la frase
                if j not in dictionary:
                    dictionary[j] = cnt
                    i[pos] = cnt
                    cnt += 1
                else:
                    i[pos] = dictionary[j]

        return dataset, cnt, dictionary 

    def labelEncoder(self):
        train, train_id, dev, dev_id, test, test_id = self.read_split()
        cnt = 1
        cnt_id = 0

        train, cnt, self.data = self._tagger(train, cnt, self.data) 
        train_id, cnt_id, self.labels = self._tagger(train_id, cnt_id, self.labels)
        dev, cnt, self.data = self._tagger(dev, cnt, self.data)
        dev_id, cnt_id, self.labels = self._tagger(dev_id, cnt_id, self.labels)
        
        len_train = 0
        for i in train:
            len_train += len(i)

        for phrase_te in test:
            for te in phrase_te:
                pos = phrase_te.index(te)
                if te not in self.data:
                    self.data[te] = len_train
                    phrase_te[pos] = self.data[te]
                else:
                    phrase_te[pos] = self.data[te]

        for phrase_te_id in test_id:
            for te_id in phrase_te_id:
                pos_id = phrase_te_id.index(te_id)
                if te_id not in self.labels:
                    self.labels[te_id] = len_train
                    phrase_te_id[pos_id] = self.labels[te_id]
                else:
                    phrase_te_id[pos_id] = self.labels[te_id] 

        return train, train_id, dev, dev_id, test, test_id, self.data, self.labels

In [40]:
train_PartTUT, train_id_PartTUT, dev_PartTUT, dev_id_PartTUT, test_PartTUT, test_id_PartTUT, PartTUT_data, PartTUT_labels = alphabet('materiales_practica/datasets/PartTUT/train.txt', 'materiales_practica/datasets/PartTUT/dev.txt', 'materiales_practica/datasets/PartTUT/test.txt').labelEncoder()

In [60]:
print(test_id_PartTUT)

[[0, 6, 7, 12, 6], [2, 0, 1, 2, 0, 9, 13, 1, 5, 1, 2, 0, 8, 0, 0, 3, 5, 6], [13, 10, 5, 1, 1, 2, 0, 1, 0, 6, 0, 5, 2, 0, 1, 6, 5, 8, 5, 2, 0, 8, 0, 1, 2, 0, 5, 2, 0, 6, 9, 6, 5, 6, 9, 8, 10, 6, 5, 6, 1, 0, 6, 0, 1, 0, 6, 0, 6, 0, 1, 2, 9, 0, 6, 0, 6, 8, 2, 0, 1, 9, 8, 9, 0, 6, 0, 6, 8, 2, 0, 1, 0, 1, 0, 6, 13, 8, 4, 9, 6], [7, 7, 3, 4, 2, 0, 1, 2, 0, 6, 8, 5, 2, 0, 9, 1, 0, 1, 2, 0, 6], [15, 6, 0, 7, 6, 11, 5, 2, 0, 1, 2, 0, 11, 3, 10, 5, 3, 3, 10, 9, 6], [11, 3, 5, 13, 11, 5, 6, 13, 11, 3, 4, 2, 0, 6], [0, 0, 3, 10, 3, 1, 2, 0, 10, 6], [9, 4, 5, 6, 0, 1, 0, 6, 0, 8, 9, 0, 3, 1, 0, 0, 8, 6, 5, 2, 9, 0, 1, 2, 0, 1, 0, 6, 0, 1, 0, 0, 3, 10, 3, 5, 6, 10, 6, 10, 10, 1, 2, 0, 1, 2, 9, 7, 6, 1, 0, 6, 11, 3, 1, 0, 6], [11, 3, 4, 10, 2, 0, 1, 2, 0, 6, 8, 11, 5, 13, 11, 5, 4, 5, 0, 10, 10, 10, 1, 1, 5, 0, 1, 2, 0, 10, 10, 6], [11, 3, 5, 1, 2, 9, 0, 6, 8, 11, 3, 5, 2, 0, 4, 5, 2, 0, 10, 10, 6], [8, 11, 5, 13, 11, 3, 5, 11, 11, 3, 4, 5, 2, 0, 1, 9, 0, 1, 2, 0, 6, 1, 2, 0, 6, 13, 11, 3, 0, 0, 8, 4

In [172]:
train_MITMovie, train_id_MITMovie, dev_MITMovie, dev_id_MITMovie, test_MITMovie, test_id_MITMovie, MITMovie_data, MITMovie_labels = alphabet('materiales_practica/datasets/MITMovie/train.txt', 'materiales_practica/datasets/MITMovie/dev.txt', 'materiales_practica/datasets/MITMovie/test.txt').labelEncoder()

In [68]:
train_MITRestaurant, train_id_MITRestaurant, dev_MITRestaurant, dev_id_MITRestaurant, test_MITRestaurant, test_id_MITRestaurant, MITRestaurant_data, MITRestaurant_labels = alphabet('materiales_practica/datasets/MITRestaurant/train.txt', 'materiales_practica/datasets/MITRestaurant/dev.txt', 'materiales_practica/datasets/MITRestaurant/test.txt').labelEncoder()

In [83]:
class FFTagger():
    def __init__(self, train, train_id, dev, dev_id, test, test_id, labels_dict, n, loss, optimizer, metrics): # weighted_metrics
        self.model = Sequential()
        self.train = train
        self.train_id = train_id
        self.dev = dev
        self.dev_id = dev_id
        self.test = test
        self.test_id = test_id
        self.labels_dict = labels_dict
        self.n = n
        len_train = 0
        for i in train:
            len_train += len(i)
        self.vocab_size = len_train + 1 # +1 por valores desconocidos en test
        self.classes = labels_dict.keys()
        self.num_classes = len(labels_dict)
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics
        # self.weighted_metrics = weighted_metrics
        self.train_windows = []
        self.dev_windows = []
        self.test_windows = []
        self.batch_size = 64

    def build_model(self): 
        self.model.add(Input(shape=(self.n*2+1,), dtype=tf.int32))
        self.model.add(Embedding(input_dim = self.vocab_size, output_dim=20, mask_zero=True, input_length=self.n*2+1))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(self.num_classes, activation='softmax'))

    def train_model(self):
        # Añadimos padding a las frases y las dividimos en ventanas de tamaño 2n+1
        padding = []
        for i in range(self.n):
            padding.append(0)

        padded_train = []
        padded_dev = []
        one_hot_train_id = []
        one_hot_dev_id = []
        for j in range(len(self.train)):
            padded_phrase = padding + self.train[j] + padding
            padded_train.append(padded_phrase) # padding
            for w in range(self.n, len(padded_phrase)-self.n): # división en ventanas
                self.train_windows.append(padded_phrase[w-self.n:w+self.n+1])
            one_hot_phrase = to_categorical(self.train_id[j], num_classes=self.num_classes) # one-hot encoding
            for v in one_hot_phrase:
                one_hot_train_id.append(v)

        for k in range(len(self.dev)):
            padded_phrase_dev = padding + self.dev[k] + padding
            padded_dev.append(padded_phrase_dev)
            for w in range(self.n, len(padded_phrase_dev)-self.n):
                self.dev_windows.append(padded_phrase_dev[w-self.n:w+self.n+1])
            one_hot_phrase_dev = to_categorical(self.dev_id[k], num_classes=self.num_classes)
            for v in one_hot_phrase_dev:
                one_hot_dev_id.append(v)
        
        # Convertimos las listas en tensores
        train_tensor = tf.data.Dataset.from_tensor_slices((self.train_windows, one_hot_train_id))
        train_tensor = train_tensor.batch(self.batch_size)
        dev_tensor = tf.data.Dataset.from_tensor_slices((self.dev_windows, one_hot_dev_id))
        dev_tensor = dev_tensor.batch(self.batch_size)

        # Reescribimos las variables globales
        self.train = padded_train
        self.dev = padded_dev

        self.model.compile(loss=self.loss, optimizer=self.optimizer, metrics=self.metrics)
        self.model.fit(train_tensor, epochs=1, validation_data=dev_tensor, verbose=1)

    def evaluate_model(self, task):
        padding = []
        for i in range(self.n):
            padding.append(0)
        
        # Añadimos el padding, dividimos en ventanas y hacemos one-hot encoding de las etiquetas
        padded_test = []
        one_hot_test_id = []
        for j in range(len(self.test)):
            padded_phrase_test = padding + self.test[j] + padding
            padded_test.append(padded_phrase_test)
            for w in range(self.n, len(padded_phrase_test)-self.n): # división en ventanas
                self.test_windows.append(padded_phrase_test[w-self.n:w+self.n+1])
            one_hot_phrase_test = to_categorical(self.test_id[j], num_classes=self.num_classes) # one-hot encoding
            for v in one_hot_phrase_test:
                one_hot_test_id.append(v)

        test_labels = [] # guarda las etiquetas como cadena de caracteres
        
        for i in self.test_id: 
            for j in i: 
                for k, v in self.labels_dict.items():
                    if j == v:
                        test_labels.append(k)
        
        one_hot_dict = dict() # Almacena la correspondencia entre one hot (valor) y etiqueta numérica (clave)
        pos_one_hot = 0
        for i in self.test_id:
            for j in range(len(i)):
                if i[j] not in one_hot_dict:
                    one_hot_dict[i[j]] = np.argmax(one_hot_test_id[pos_one_hot]) # devuelve el índice del elemento con valor 1 en one_hot_test_id[i]
                pos_one_hot += 1

        test_tensor = tf.data.Dataset.from_tensor_slices((self.test_windows, one_hot_test_id))
        test_tensor = test_tensor.batch(self.batch_size)
        
        if task == "PoS":
            loss, accuracy = self.model.evaluate(test_tensor, verbose=1)
            return loss, accuracy
        elif task == "NER":
            loss, accuracy = self.model.evaluate(test_tensor, verbose=1)
            pred = self.model.predict(test_tensor).astype(np.float32)

            # para cada elemento de pred obtenemos la etiqueta numérica  
            predictions = []
            for p in pred:
                pos = np.argmax(p)
                for k, v in one_hot_dict.items():
                    if pos == v:
                        predictions.append(k)
            
            # convertimos las etiquetas numéricas a etiquetas de texto
            pred_labels = []
            for l in predictions:
                for k, v in self.labels_dict.items():
                    if l == v:
                        pred_labels.append(k)

            numeric_tags = set()
            for i in self.train_id:
                numeric_tags = set(numeric_tags|set(i))
            for j in self.dev_id:
                numeric_tags = set(numeric_tags|set(j))
            numeric_tags = list(numeric_tags)

            # convertimos las etiquetas numéricas a etiquetas de texto
            tags = []
            for t in numeric_tags:
                for k, v in self.labels_dict.items():
                    if t == v:
                        tags.append(k)

            # Nos quedamos con las etiquetas de la entidad (sin B e I)
            evaluator_tags = []
            for tag in tags:
                if tag == "O":
                    evaluator_tag = tag
                else:
                    evaluator_tag = tag[2:]

                if evaluator_tag not in evaluator_tags:
                    evaluator_tags.append(evaluator_tag)
                    
            evaluator = Evaluator(test_labels, pred=pred_labels, tags=evaluator_tags, loader="list")
            results, results_by_tag = evaluator.evaluate()
            
            return loss, accuracy, results, results_by_tag
        else:
            return "Task not found"

In [66]:
modelPartTUT = FFTagger(train_PartTUT, train_id_PartTUT, dev_PartTUT, dev_id_PartTUT, test_PartTUT, test_id_PartTUT,  PartTUT_labels, 2, 'categorical_crossentropy', 'adam', ['accuracy'])
modelPartTUT.build_model()
modelPartTUT.train_model()

680/680 [==============================] - 10s 13ms/step - loss: 1.1826 - accuracy: 0.6725 - val_loss: 0.6220 - val_accuracy: 0.8051


In [67]:
modelPartTUT.evaluate_model("PoS")

53/53 [==============================] - 0s 1ms/step - loss: 0.5380 - accuracy: 0.8398


(0.5380165576934814, 0.8398101329803467)

In [174]:
modelMITMovie = FFTagger(train_MITMovie, train_id_MITMovie, dev_MITMovie, dev_id_MITMovie, test_MITMovie, test_id_MITMovie, MITMovie_labels, 2, 'categorical_crossentropy', 'adam', ['accuracy'])
modelMITMovie.build_model()
modelMITMovie.train_model()

14283/14283 [==============================] - 914s 64ms/step - loss: 0.7141 - accuracy: 0.7890 - val_loss: 0.5518 - val_accuracy: 0.8295


In [183]:
# modelMITMovie.evaluate_model("NER")

In [84]:
modelMITRestaurant = FFTagger(train_MITRestaurant, train_id_MITRestaurant, dev_MITRestaurant, dev_id_MITRestaurant, test_MITRestaurant, test_id_MITRestaurant, MITRestaurant_labels, 2, 'categorical_crossentropy', 'adam', ['accuracy'])
modelMITRestaurant.build_model()
modelMITRestaurant.train_model()

993/993 [==============================] - 20s 19ms/step - loss: 0.9874 - accuracy: 0.7401 - val_loss: 0.5572 - val_accuracy: 0.8487


In [85]:
modelMITRestaurant.evaluate_model("NER")

223/223 [==============================] - 0s 1ms/step


(0.5848591327667236,
 0.8444974422454834,
 {'ent_type': {'correct': 4937,
   'incorrect': 0,
   'partial': 0,
   'missed': 660,
   'spurious': 399,
   'possible': 5597,
   'actual': 5336,
   'precision': 0.9252248875562219,
   'recall': 0.8820796855458282,
   'f1': 0.9031372907710601},
  'partial': {'correct': 4061,
   'incorrect': 0,
   'partial': 876,
   'missed': 660,
   'spurious': 399,
   'possible': 5597,
   'actual': 5336,
   'precision': 0.8431409295352323,
   'recall': 0.8038234768626049,
   'f1': 0.8230128967346564},
  'strict': {'correct': 4061,
   'incorrect': 876,
   'partial': 0,
   'missed': 660,
   'spurious': 399,
   'possible': 5597,
   'actual': 5336,
   'precision': 0.7610569715142429,
   'recall': 0.7255672681793818,
   'f1': 0.742888502698253},
  'exact': {'correct': 4061,
   'incorrect': 876,
   'partial': 0,
   'missed': 660,
   'spurious': 399,
   'possible': 5597,
   'actual': 5336,
   'precision': 0.7610569715142429,
   'recall': 0.7255672681793818,
   'f1': 

In [19]:
class LSTMTagger():
    def __init__(self, train, train_id, dev, dev_id, test, test_id, data_dict, labels_dict, loss, optimizer, metrics):
        self.model = Sequential()
        self.train = train
        self.train_id = train_id
        self.dev = dev
        self.dev_id = dev_id
        self.test = test
        self.test_id = test_id
        self.data_dict = data_dict
        self.labels_dict = labels_dict
        self.vocab_size = len(self.train) + 1
        self.classes = set(set(self.train_id) | set(self.dev_id))
        self.num_classes = len(self.classes)
        self.loss=loss
        self.optimizer=optimizer
        self.metrics=metrics
        self.train_windows = []
        self.train_windows_id = []
        self.dev_windows = []
        self.dev_windows_id = []
        self.test_windows = []
        self.test_windows_id = []
        self.point_id = self.data_dict["."]
        self.question_mark_id = self.data_dict["?"]
        self.exclamation_mark_id = self.data_dict["!"]
        self.maxlen = 0
        self.batch_size = 64

    def build_model(self, bidirectional=False):
        self.model.add(Embedding(input_dim=self.vocab_size, output_dim=20, mask_zero=True, input_length=self.maxlen))
        if bidirectional:
            self.model.add(Bidirectional(LSTM(64, return_sequences=True)))
        else:
            self.model.add(LSTM(64, return_sequences=True))
        self.model.add(TimeDistributed(Dense(units=self.num_classes, activation='softmax')))

    def _window_maker(self, data, data_id, windows, windows_id):
        window = []
        window_id = []
        # Separamos en oraciones
        for i in range(len(data)):
            window.append(data[i])
            window_id.append(data_id[i])
            if data[i] == self.point_id or data[i] == self.question_mark_id or data[i] == self.exclamation_mark_id:
                windows.append(window)
                windows_id.append(window_id)
                if len(window) > self.maxlen:
                    self.maxlen = len(window)
                window = []
                window_id = []

        return windows, windows_id    
        
    def _padder(self, data, data_id):
        padded_data = pad_sequences(data, maxlen=self.maxlen, padding='post')
        padded_data_id = pad_sequences(data_id, maxlen=self.maxlen, padding='post')
        one_hot_data_id = to_categorical(padded_data_id, num_classes=self.num_classes)

        return padded_data, one_hot_data_id
          

    def preprocessing(self):
        self.train_windows, self.train_windows_id = self._window_maker(self.train, self.train_id, self.train_windows, self.train_windows_id)
        self.dev_windows, self.dev_windows_id = self._window_maker(self.dev, self.dev_id, self.dev_windows, self.dev_windows_id)
        self.test_windows, self.test_windows_id = self._window_maker(self.test, self.test_id, self.test_windows, self.test_windows_id)

        self.train_windows, self.train_windows_id = self._padder(self.train_windows, self.train_windows_id)
        self.dev_windows, self.dev_windows_id = self._padder(self.dev_windows, self.dev_windows_id)
        self.test_windows, self.test_windows_id = self._padder(self.test_windows, self.test_windows_id)

    def train_model(self):
        train_tensor = tf.data.Dataset.from_tensor_slices((self.train_windows, self.train_windows_id))
        train_tensor = train_tensor.batch(self.batch_size)
        dev_tensor = tf.data.Dataset.from_tensor_slices((self.dev_windows, self.dev_windows_id))
        dev_tensor = dev_tensor.batch(self.batch_size)

        self.model.compile(loss=self.loss, optimizer=self.optimizer, metrics=self.metrics)
        self.model.fit(train_tensor, epochs=1, validation_data=dev_tensor)

    def evaluate_model(self, task):
        test_labels = [] # lista que almacena las etiquetas de los elementos de self.test_id
        
        for i in self.test_id:
            for k, v in self.labels_dict.items():
                if i == v:
                    test_labels.append(k)
        
        one_hot_dict = dict()
        cnt_test_id = 0

        for w in self.test_windows_id:
            for i in range(len(w)):
                if np.argmax(w[i]) not in one_hot_dict:
                    one_hot_dict[np.argmax(w[i])] = self.test_id[cnt_test_id] # clave es el índice del elemento con valor 1 en one_hot_test_id[i], valor es el elemento de self.test_id -> al reves que en FFTagger

                    cnt_test_id += 1

        test_tensor = tf.data.Dataset.from_tensor_slices((self.test_windows, self.test_windows_id))
        test_tensor = test_tensor.batch(self.batch_size)
        
        if task == "PoS":
            loss, accuracy = self.model.evaluate(test_tensor, verbose=1)
            return loss, accuracy
        elif task == "NER":
            loss, accuracy = self.model.evaluate(test_tensor, verbose=1)
            pred = self.model.predict(test_tensor).astype(np.float32)

##############################################################
            predictions = []
            # Nos quedamos con el valor de los paddings
            for k, v in one_hot_dict.items():
                if v == 0:
                    pos_0 = k
            for phrase in pred:
                for i in range(len(phrase)):
                    pos = np.argmax(phrase[i])
                    if pos != pos_0: # si es un valor no padding
                        for k, v in one_hot_dict.items():
                            if pos == k:
                                predictions.append(v)
###############################################################

            numeric_tags = list(set(set(self.train_id) | set(self.dev_id)))

            # convertimos las etiquetas numéricas a etiquetas de texto
            tags = []
            for t in numeric_tags:
                for k, v in self.labels_dict.items():
                    if t == v:
                        tags.append(k)

            evaluator_tags = []
            for tag in tags:
                if tag == "O":
                    evaluator_tag = tag
                else:
                    evaluator_tag = tag[2:]

                if evaluator_tag not in evaluator_tags:
                    evaluator_tags.append(evaluator_tag)

            pred_labels = []
            for l in predictions:
                for k, v in self.labels_dict.items():
                    if l == v:
                        pred_labels.append(k)

            print(len(test_labels), test_labels[0:100])
            print(len(pred_labels), pred_labels[0:100])

            evaluator = Evaluator(test_labels, pred=pred_labels, tags=evaluator_tags, loader="list")
            results, results_by_tag = evaluator.evaluate()
            
            return loss, accuracy, results, results_by_tag
        else:
            return "Task not found"

In [20]:
LSTMPartut = LSTMTagger(train_PartTUT, train_id_PartTUT, dev_PartTUT, dev_id_PartTUT, test_PartTUT, test_id_PartTUT, PartTUT_data, PartTUT_labels, 'categorical_crossentropy', 'adam', ['accuracy'])
LSTMPartut.preprocessing()
LSTMPartut.build_model(bidirectional=True)
LSTMPartut.train_model()

27/27 [==============================] - 18s 453ms/step - loss: 0.2266 - accuracy: 0.2013 - val_loss: 0.1449 - val_accuracy: 0.2083


In [21]:
LSTMPartut.evaluate_model("NER")

3/3 [==============================] - 0s 62ms/step - loss: 0.1757 - accuracy: 0.2212


2023-04-25 11:41:41 tensorflow WARNING: 5 out of the last 13 calls to <function Model.make_predict_function.<locals>.predict_function at 0x000002800A41F520> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


3/3 [==============================] - 2s 69ms/step
3408 ['NOUN', 'PUNCT', 'PROPN', 'NUM', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'SCONJ', 'ADP', 'VERB', 'ADP', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'NOUN', 'AUX', 'VERB', 'PUNCT', 'SCONJ', 'ADV', 'VERB', 'ADP', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'NOUN', 'VERB', 'DET', 'NOUN', 'ADP', 'PUNCT', 'VERB', 'CCONJ', 'VERB', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'PUNCT', 'ADJ', 'PUNCT', 'VERB', 'PUNCT', 'ADJ', 'CCONJ', 'ADV', 'PUNCT', 'VERB', 'PUNCT', 'ADP', 'NOUN', 'PUNCT', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'CCONJ', 'DET', 'NOUN', 'ADP', 'ADJ', 'CCONJ', 'ADJ', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'CCONJ', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'SCONJ']
47414 ['NOUN', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'PUNCT', 'PUNCT', 'PUNCT', 'PUNCT', 'PUNCT', 'PUNCT', 'PUNCT', 'PUNCT', 'PUNCT', 'PUNCT', 'PUNCT', 'P

ValueError: Number of predicted documents does not equal true